In [1]:
from scipy.sparse import hstack
import pickle
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
import joblib
from elasticsearch import Elasticsearch, helpers
import time
import pandas as pd
import numpy as np


In [4]:
user_df = pd.read_parquet("resources/food/user.parquet")
user_df


,username,password,display_name
0,test,test,test-user
1,test_register,test_register,test_register
2,test2,test2,Jamie Oliver
3,test3,test3,Gorgon Ramsey


In [36]:
recipes_df = pd.read_parquet("resources/food/recipes.parquet")
recipes_df


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379.0,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,PT35M,PT1H,PT1H35M,2020-12-22 15:27:00+00:00,Make and share this Meg's Fresh Ginger Gingerb...,[],...,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,[Preheat oven to 350&deg;F Grease an 8x8 cake ...
522513,541380.0,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,PT3H,PT30M,PT3H30M,2020-12-22 15:32:00+00:00,"White, black, green, and pink peppercorns add ...",[https://img.sndimg.com/food/image/upload/w_55...,...,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,[Position rack in center of oven and preheat t...
522514,541381.0,Kirshwasser Ice Cream,2001131545,Jonathan F.,PT3H,PT1H,PT4H,2020-12-22 15:33:00+00:00,Make and share this Kirshwasser Ice Cream reci...,[],...,72.6,470.9,192.5,33.9,0.0,17.3,12.8,6.0,None,[heat half and half and heavy cream to a simme...
522515,541382.0,Quick & Easy Asian Cucumber Salmon Rolls,2001004241,CLUBFOODY,None,PT15M,PT15M,2020-12-22 22:11:00+00:00,"Extremely quick and easy to make, these are gr...",[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,2.9,100.5,0.3,0.0,0.1,2.4,NaN,20 rolls,"[In a small bowl, combine mayo and wasabi past..."


In [55]:
folder_df = pd.DataFrame([["test", "Beef Dishes"], ["test", "Asian Dishes"], ["test", "Monday Mood"]], columns=["username", "folder_name"])
folder_df


,username,folder_name
0,test,Beef Dishes
1,test,Asian Dishes
2,test,Monday Mood


In [68]:
bookmark_df = pd.DataFrame([[0, 27208, 5.0], [1, 45809, 5.0], [2, 89204, 4.0], [2, 39087, 5.0]], columns=["folder_index", "RecipeID", "rating"])
bookmark_df.to_parquet("resources/food/bookmark.parquet")


In [47]:
def get_folder_username(username: string):
    return folder_df[folder_df["username"] == username]
get_folder_username("test")


,username,folder_name
0,test,Beef Dishes
1,test,Asian Dishes
2,test,Monday Mood


In [80]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

app = dotdict()
app.folder_df = pd.read_parquet("resources/food/folder.parquet")
app.bookmark_df = pd.read_parquet("resources/food/bookmark.parquet")


In [81]:
def create_folder(username: string, folder_name: string):
    app.folder_df = pd.concat([app.folder_df, pd.DataFrame([[username, folder_name]], columns=["username", "folder_name"])], ignore_index=True)
    app.folder_df.to_parquet("resources/food/folder.parquet")
    return app.folder_df
create_folder("test3", "OPPENHEIMER")


,username,folder_name
0,test,Beef Dishes
1,test,Asian Dishes
2,test,Monday Mood
3,test2,POWER
4,test3,OPPENHEIMER
5,test3,OPPENHEIMER


In [100]:
def get_recipes_folder(id: int):
    return app.bookmark_df.loc[app.bookmark_df["folder_index"] == id].join(recipes_df, on=["RecipeID"])[["rating"] + list(recipes_df.columns)]

get_recipes_folder(3)


,rating,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
4,4.0,78.0,Buttermilk Ranch Dressing,1582,Diana van den Broek,None,PT20M,PT20M,1999-08-18 04:20:00+00:00,Make and share this Buttermilk Ranch Dressing ...,...,12.8,46.4,577.9,14.1,0.3,7.7,8.2,NaN,1 1/4 cups,[Mash together garlic and salt to form a paste...
5,3.0,79.0,Brisket with Sauerkraut and Applesauce,1549,Dave5003,PT3H,PT25M,PT3H25M,1999-09-06 04:33:00+00:00,Make and share this Brisket with Sauerkraut an...,...,5.4,36.5,475.7,19.3,2.3,8.5,9.6,6.0,None,"[In a large cook pot place undrained tomatoes,..."


In [99]:
def bookmark_recipe(folder_index: int, RecipeID: int, rating: float):
    if (not app.bookmark_df.loc[(app.bookmark_df["folder_index"] == folder_index) & (app.bookmark_df["RecipeID"] == RecipeID)].empty):
        app.bookmark_df.loc[(app.bookmark_df["folder_index"] == folder_index) & (app.bookmark_df["RecipeID"] == RecipeID), "rating"] = rating
    else:
        app.bookmark_df = pd.concat([app.bookmark_df, pd.DataFrame([[folder_index, RecipeID, rating]], columns=["folder_index", "RecipeID", "rating"])], ignore_index=True)
    return app.bookmark_df
bookmark_recipe(3, 39, 3.0)


,folder_index,RecipeID,rating
0,0,27208,5.0
1,1,45809,5.0
2,2,89204,4.0
3,2,39087,5.0
4,3,38,4.0
5,3,39,3.0
